# <font color='#31708f'><center>CountVectorizer</center></font>

In [4]:
import glob

In [5]:
file_list = glob.glob('C:/Users/smsta/Desktop/GIT_REPOSITORIES/zendesk_tag_cloud/newman/newman-run-report-2021-02-09-13-53-48-837-0-ticketid-upto-59963.json')
for filename in file_list:
    print(filename)

C:/Users/smsta/Desktop/GIT_REPOSITORIES/zendesk_tag_cloud/newman/newman-run-report-2021-02-09-13-53-48-837-0-ticketid-upto-59963.json


In [ ]:
import ijson
def parse_json(json_filename):
        with open(filename, 'r', encoding="utf8") as file:
            # load json iteratively
            parser = ijson.parse(file)
            for prefix, event, value in parser:
                print('prefix={}, event={}, value={}'.format(prefix, event, value))
    
if __name__ == '__main__':
    parse_json(filename)

In [6]:
import ijson
import re
from string import punctuation
def extract_ticket_text_generator(json_filename):
    """This function takes a list of files with tickets and extracts text from each ticket. The result is a list of text strings."""
    for filename in file_list:
            with open(filename, 'r', encoding="utf8") as input_file:
                # Extract specific items from the file
                tickets = ijson.items(input_file, 'run.executions.item.assertions.item.assertion')
                for ticket in tickets:
                    # Extract the substring between two markers
                    l = re.findall('plain_body(.+?)public', ticket)
                    # Remove all of the punctuation in any item in the list. The result is for each ticket a list of comments.
                    m = [''.join(c for c in s if c not in punctuation) for s in l]
                    # Join list elements without any separator. The result is for each ticket a list of merged comments.
                    n = [' '.join(m)] 
                    yield(n)
                
    if __name__ == '__main__':
        extract_ticket_text_generator(filename)

In [12]:
def create_txt_files():
        """This function takes a list of text strings and saves each ticket in a .txt file."""
        data = extract_ticket_text_generator(filename)
        # Make a flat list out of list of lists.
        flat_list = [item for sublist in data for item in sublist]
        for i in range(len(flat_list)):
            with open("ticket_%d.txt" % (i+59047), 'w', encoding="utf-8") as f:
                f.write(flat_list[i])

In [13]:
create_txt_files()

Outside of project scope - create a .txt file with all tickets in it:

In [14]:
def create_tickets_all_txt_file():
        """This function takes a list of text strings and saves all tickets in a .txt file."""
        data = extract_ticket_text_generator(filename)
        # Make a flat list out of list of lists.
        flat_list = [item for sublist in data for item in sublist]
        with open('ticket_all_lockdown.txt', 'w', encoding="utf-8") as filehandle:
            #Save all elements of a list as a text file:
            for listitem in flat_list:
                filehandle.write('%s\n\n' % listitem)

In [15]:
create_tickets_all_txt_file()